In [1]:
import os
import gc
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *

% reload_ext autoreload
% autoreload 2

In [27]:
PATH = os.path.abspath(r"C:/Users/rsilva/Desktop/SEI/")
print(PATH)               

C:\Users\rsilva\Desktop\SEI


In [8]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [9]:
sei_ = sei.login_sei(browser, usr, pwd)

In [10]:
sei_ = sei.Sei(sei_)

In [11]:
sei_.itera_processos()

In [6]:
scpx = webdriver.Ie()
scpx = sistemas.Scpx(scpx)

In [15]:
cpf = '14804020870'
proc = r'53504.008508/2018-89'

In [48]:
p = sei_.go_to_processo(proc)

In [34]:
scpx.consulta(cpf)

In [36]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2J1510')

In [37]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc)

In [38]:
scpx.licenciar_estacao(cpf)

In [39]:
scpx.licenciar_estacao(cpf, ppdess=False)

In [40]:
files = sorted(os.listdir(PATH), reverse=True)
file = os.path.join(PATH, files[0])

In [47]:
p.incluir_doc_externo('Boleto', file)

In [49]:
p.incluir_oficio('RC_PPDESS')

ValueError: Tipo de Ofício inválido: RC_PPDESS